In [1]:
print("Ok")

Ok


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\Satish\.conda\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
PINECONE_API_KEY = "77f3280e-ab97-41f0-815b-260b12cbdde4"
PINECONE_API_ENV = "gcp-starter"

In [9]:
#Extract data from pdf
def load_pdf(data):
    loader = DirectoryLoader(data, glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [8]:
!pip install pypdf

  Using cached pypdf-4.2.0-py3-none-any.whl.metadata (7.4 kB)
Using cached pypdf-4.2.0-py3-none-any.whl (290 kB)


In [10]:
extracted_data = load_pdf("data/")

In [12]:
#extracted_data

In [13]:
# create text chunks

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [14]:
#each chunk 500 tokens, total chunks = 7020
text_chunks = text_split(extracted_data)
print("length of chunk:", len(text_chunks))

length of chunk: 7020


In [15]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [16]:
embeddings = download_hugging_face_embeddings()

In [17]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [18]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


In [34]:
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medical-chatbot"
index = pc.Index(index_name)

In [40]:
!pip install pinecone-client

In [41]:
from langchain.vectorstores import Pinecone as PC

pinecone_index = PC.from_texts(text_chunks, embeddings, index_name=index_name)

AttributeError: list_indexes is no longer a top-level attribute of the pinecone package.

To use list_indexes, please create a client instance and call the method there instead.

Example:

    from pinecone import Pinecone
    
    pc = Pinecone(api_key='YOUR_API_KEY')

    index_name = "quickstart" # or your index name

    if index_name not in pc.list_indexes().names():
        # do something



In [44]:
import pinecone

# Assuming you have defined 'text_chunks', 'embeddings', and 'index_name' variables

# Create a Pinecone client
client = pinecone.client()

# Create a Pinecone index from text data
index = client.create_index(index_name)

# Extract text content from text chunks
text_content_list = [t.page_content for t in text_chunks]

# Index the text content with their corresponding embeddings
index.upsert(text_content_list, embeddings)

# Close the Pinecone index
#index.delete()



AttributeError: module 'pinecone' has no attribute 'client'

In [37]:
docsearch = pinecone.from_text([t.page_content for t in text_chunks], embeddings, index_name=index_name)

AttributeError: module 'pinecone' has no attribute 'from_text'

In [30]:
#Pinecone.init(api_key = PINECONE_API_KEY,
             # environment = PINECONE_API_ENV)

pc = Pinecone(api_key='PINECONE_API_KEY')
index = pc.Index('medical-chatbot')


index_name = "medical-chatbot"

docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'X-Cloud-Trace-Context': '483ca3787019a20f50bb8f8a4c311ded', 'Date': 'Wed, 24 Apr 2024 17:07:17 GMT', 'Content-Type': 'text/html', 'Server': 'Google Frontend', 'Content-Length': '15', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Invalid API Key
